In [ ]:
import glob,os 
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
import rioxarray as xrx
import pandas as pd, numpy as np
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely.validation import make_valid

In [ ]:
top_dir = '/datawaha/esom/Sentinel_2/DeliRsl/RegionAll_afclean'
tile_keys = set([ifile.split('_')[-2] for ifile in os.listdir(top_dir)])
tile_keys = sorted(list(tile_keys))
tile_keys[0:4]

In [ ]:

def _fill_holes(geom):
    """Return geometry with all interior rings removed."""
    if geom.is_empty:
        return geom
    geom = make_valid(geom)
    if isinstance(geom, Polygon):
        return Polygon(geom.exterior)
    if isinstance(geom, MultiPolygon):
        return MultiPolygon([Polygon(p.exterior) for p in geom.geoms if not p.is_empty])
    # For any other type (rare after polygonize), just return as-is
    return geom


def polygonize_fd_id(xr_tif, target_fd_type, dissolve=True):
    """
    Create polygons for each unique fd_id where fd_type == target_fd_type,
    filling holes within each polygon.

    Parameters
    ----------
    xr_tif : xarray.Dataset
        Raster with bands 'fd_type' and 'fd_id'
    target_fd_type : int
        The fd_type value to extract (e.g., 1, 2, 3...)
    dissolve : bool
        If True, ensures one polygon per unique fd_id

    Returns
    -------
    gpd.GeoDataFrame
        Polygons for all fd_id matching the fd_type filter (hole-free)
    """

    # Extract arrays
    fd_type = xr_tif["fd_type"].values
    fd_id = xr_tif["fd_id"].values
    transform = xr_tif.rio.transform()
    crs = xr_tif.rio.crs

    # Mask: keep only matching fd_type
    mask = (fd_type == target_fd_type)

    # Apply mask to fd_id
    masked_fd_id = fd_id.copy()
    masked_fd_id[~mask] = 0  # background

    # Polygonize
    polygons = []
    ids = []
    for geom, value in shapes(masked_fd_id, mask=mask, transform=transform):
        if value == 0:
            continue
        polygons.append(shape(geom))
        ids.append(int(value))

    # Make GeoDataFrame
    gdf = gpd.GeoDataFrame({"fd_id": ids, "geometry": polygons}, crs=crs)

    # Remove holes before dissolve (cleanup small artifacts early)
    gdf["geometry"] = gdf.geometry.apply(_fill_holes)

    # Optional dissolve (one polygon per fd_id)
    if dissolve:
        gdf = gdf.dissolve(by="fd_id", as_index=False)
        # Remove holes again in case unioning reintroduced interiors
        gdf["geometry"] = gdf.geometry.apply(_fill_holes)

    # Final validity pass
    gdf["geometry"] = gdf.geometry.apply(make_valid)

    return gdf

# Get polygons where fd_type == 1
def polygonize_all_fd_types(xr_tif, tile_key, year_key):
    gdf1 = polygonize_fd_id(xr_tif, target_fd_type=1)
    gdf1['field_type'] = 'circles'
    gdf2 = polygonize_fd_id(xr_tif, target_fd_type=2)
    gdf2['field_type'] = 'fans'
    gdf5 = polygonize_fd_id(xr_tif, target_fd_type=5)
    gdf5['field_type'] = 'merged'

    # Suppose you have a list of GeoDataFrames
    gdfs = [gdf1, gdf2, gdf5]   # add as many as you want
    # Concatenate
    gdf_all = pd.concat(gdfs, ignore_index=True)
    # Ensure GeoDataFrame identity remains intact
    gdf_all = gpd.GeoDataFrame(gdf_all, geometry="geometry", crs=gdfs[0].crs)
    gdf_all['fd_id'] = np.arange(1, len(gdf_all)+1)
    gdf_all['Satellite Tile'] = tile_key 
    gdf_all['Year'] = year_key
    # now add the field acreage 
    gdf_all['Acreage_m2'] = gdf_all.geometry.area
    gdf_all['Acreage_ha'] = gdf_all['Acreage_m2'] / 10000.0
    # round to 2 decimal places
    gdf_all['Acreage_ha'] = gdf_all['Acreage_ha'].round(2)
    # reporject to EPSG:4326
    gdf_all = gdf_all.to_crs(epsg=4326)
    # now save to shapefile
    return gdf_all



In [ ]:
from tqdm.auto import tqdm
years = ['2019', '2020', '2021', '2022', '2023']

with tqdm(total=len(years), desc="Years", position=0) as year_pbar:
    for year_key in years:
        gdf_all = []

        # inner progress bar for tiles in this year
        with tqdm(total=len(tile_keys), desc=f"{year_key} tiles", position=1, leave=False) as tile_pbar:
            for tile_key in tile_keys:
                # find raster for this (tile, year)
                matches = glob.glob(f"{top_dir}/*{tile_key}_{year_key}*")
                if not matches:
                    tile_pbar.write(f"⚠️ Missing raster for tile={tile_key}, year={year_key}")
                    tile_pbar.update(1)
                    continue

                tif_ras = matches[0]
                tile_pbar.set_postfix_str(f"tile={tile_key}")

                xr_tif = xrx.open_rasterio(
                    tif_ras, band_as_variable=True
                ).squeeze().rename({"band_1": "fd_type", "band_2": "fd_id"})

                out_gdf = polygonize_all_fd_types(xr_tif, tile_key, year_key)
                gdf_all.append(out_gdf)

                tile_pbar.update(1)

        # merge and save for this year (if anything was produced)
        if gdf_all:
            gdf_all = pd.concat(gdf_all, ignore_index=True)
            gdf_all = gpd.GeoDataFrame(gdf_all, geometry="geometry", crs=gdf_all.crs)

            out_geojson = f"/datawaha/esom/DatePalmCounting/Geoportal/Center_pivot/year_base_consistent/CPF_fields_{year_key}.geojson"
            #gdf_all.to_file(out_shp)
            gdf_all.to_file(out_geojson, driver="GeoJSON")
            tqdm.write(f"✅ Saved: {out_geojson}")
        else:
            tqdm.write(f"ℹ️ No data to save for year {year_key}")

        year_pbar.update(1)